### Sample Diabetes Classification Model Deployment
This notebook is the second part for a two-part solution, it demonstrates deployment of a trained Machine Learning Model to Azure Kubernetes cluster from a Docker Image which was created in the first notebook <a href="./build-model.ipynb">build-model</a>.

##### Azure Machine Learning and Power BI Integration

The way the scoring file was implemented the deployed web service can be consumed in Power BI in addition to any other REST client like Postman or a custom application. In order to consume the web service from Power BI DataFlows, user will need to be given permissions on the Azure Machine Learning Workspace, the following link <a href="https://docs.microsoft.com/en-us/power-bi/service-machine-learning-integration">Azure Machine Learning integration in Power BI</a> documents the process to consume the service. Please note at the time this notebook was created (June 2019), the user needs to be given Contributor level permissions  on Azure Machine Learning Workspace in contrary to what's documentated on the above link but its expected that the issue will be fixed and the official documentation will be accurate in future.

**Power BI to AML Web Service Authentication**

AML Web Service authentication is Api Key based but when configuring Power BI DataFlow to consume Web Service you will notice there is no prompt which asks of Api Key. This is because Power BI is a specialized client and once the user creating DataFlow has appropriate permissions on AML Workspace Power BI will make ListKeys call to Azure Resource Manager (which is the Control Pane) to retrieve the keys and then use the keys to communicate with the web service.  Such List Keys calls are recorded in the Activity Log.



In [9]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.image import Image
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice


## Initialization
Initialize the connection to AML Workspace, the first notebook saves the configuration in a file hence saved configuration file is used to login to Workspace

In [10]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

demoaml
demo-aml-use
eastus
36cfc6d6-79ca-4642-b263-93d6eaa4a823


In [22]:
akscompute_cluster_name = "k8cpucluster2"
image_name = "diabclassprob"
service_name = "diabdemo2"

### Deployment Target

Trained Model Web Service is deployed to an AKS Cluster (deployment targets are also referred to as Compute Target in AML terminology). The AML Workspace is queried for a list of existing Deployment Targets, an existing cluster is used if one exists (determined based on the Compute Target and Type) otherwise a new cluster is created.

**One important thing to note in the AKS Cluster creation configuration is that SSL is enabled on the cluster so that service communication is over HTTPS, in this case a certificate from Microsoft is used. Please refer to <a href="https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service#enable">AML - Secure web services with SSL</a> default configuration does not enable SSL


In [23]:
found = False

# Check if this compute target already exists in the workspace.

cts = ws.compute_targets

if akscompute_cluster_name in cts and cts[akscompute_cluster_name].type == 'AKS':
    found = True
    print('Found existing compute target.')
    deployment_target = cts[akscompute_cluster_name]

if not found:
    print('Creating a new compute target...')
        
    prov_config = AksCompute.provisioning_configuration()
    prov_config.enable_ssl(leaf_domain_label = service_name)
    
    # Create the cluster
    deployment_target = ComputeTarget.create(workspace = ws, 
                                      name = akscompute_cluster_name, 
                                      provisioning_configuration = prov_config)

    deployment_target.wait_for_completion(show_output = True)

    print(deployment_target.provisioning_state)
    print(deployment_target.provisioning_errors)

Found existing compute target.


In [29]:
akscompute_cluster_name

'k8cpucluster2'

In [24]:
image_list = Image.list(workspace = ws, image_name = image_name)

In [25]:
image = image_list[0]
print("Name: ", image.name, ",Version: ", image.version)

Name:  diabclassprob ,Version:  6


In [26]:
aks_config = AksWebservice.deploy_configuration(num_replicas=2)

In [ ]:
%%time
aks_service_name = service_name

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = deployment_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running.

In [17]:
aks_service.get_logs()

Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Tue, 25 Jun 2019 20:26:26 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'x-ms-client-request-id': '785d2ccfb2a04a8b821b5858284bbfae', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid","details":[{"code":"NoPodsAvailableForLogs","message":"There are no backend pods available for this service. No logs could be retrieved."}]}'



WebserviceException: Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Tue, 25 Jun 2019 20:26:26 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'x-ms-client-request-id': '785d2ccfb2a04a8b821b5858284bbfae', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid","details":[{"code":"NoPodsAvailableForLogs","message":"There are no backend pods available for this service. No logs could be retrieved."}]}'

### Deployed Service

* Writes out the Scoring Uri, Swagger Uri  and Api Key for Deployed Service.
* The deployed service for inferencing is a REST service and uses Api Key for authentication

In [ ]:
print("ScoringUri: ", aks_service.scoring_uri)
print("Swagger Uri: ", aks_service.swagger_uri)
key1,key2 = aks_service.get_keys()
print(key1)

In [ ]:
# construct raw HTTP request and send to the service
import requests
import json

test_sample = json.dumps({'data': [
    	{
					"pregnancies": "6",
	    			"plasma glucose": "148",
	    			"blood pressure": "72",
	    			"triceps skin thickness": "35",
	    			"insulin": "0",
	    			"bmi": "33.6",
	    			"diabetes pedigree": "0.627",
	    			"age": "50"
				},
				{
					"pregnancies": "1",
	    			"plasma glucose": "85",
	    			"blood pressure": "66",
	    			"triceps skin thickness": "29",
	    			"insulin": "0",
	    			"bmi": "26.6",
	    			"diabetes pedigree": "0.351",
	    			"age": "31"
				}
]})
test_sample = bytes(test_sample,encoding = 'utf8')

# Don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)


print("prediction:", resp.text)
